In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
path = '../input/store-sales-time-series-forecasting/'

sub = pd.read_csv(path + 'sample_submission.csv')

holiday_df = pd.read_csv(path + 'holidays_events.csv')
oil_df = pd.read_csv(path + 'oil.csv')
test_df = pd.read_csv(path + 'test.csv')
tran_df = pd.read_csv(path + 'transactions.csv')
train_df = pd.read_csv(path + 'train.csv')
store_df = pd.read_csv(path + 'stores.csv')
#sales_df = pd.read_csv(path + 'sales.csv')
#cities_df = pd.read_csv(path + 'cities.csv')
#product_lines_df = pd.read_csv('product_lines.csv')

In [3]:
# Load the CSV files into pandas DataFrames

# Merge the necessary information from different DataFrames for total sales
merged_data = pd.merge(train_df, store_df, on='store_nbr')

# Calculate the total sales for each store
store_sales = train_df.groupby('store_nbr')['sales'].sum().sort_values(ascending=False)
#store_sales = data.groupby('store_nbr')['sales'].sum()


# Get the top 5 stores with the highest total sales
top_5_stores = store_sales.head(5)

# Calculate the total sales for each city
city_sales = merged_data.groupby('city')['sales'].sum().sort_values(ascending=False)


# Get the top 5 cities with the highest total sales
top_5_cities = city_sales.head(5)

# Calculate the total sales for each product family
family_sales = train_df.groupby('family')['sales'].sum()

# Sort the product families based on sales in descending order
sorted_family_sales = family_sales.sort_values(ascending=False)

# Get the top 10 families with the most sales
top_10_families = sorted_family_sales.head(10)



# Print the results
print("Top 5 stores by total sales:")
print(top_5_stores)
print("\nTop 5 cities by total sales:")
print(top_5_cities)
print("\nTop 10 product lines by total sales:")
print(top_10_families)


Top 5 stores by total sales:
store_nbr
44    6.208755e+07
45    5.449801e+07
47    5.094831e+07
3     5.048191e+07
49    4.342010e+07
Name: sales, dtype: float64

Top 5 cities by total sales:
city
Quito            5.567418e+08
Guayaquil        1.229673e+08
Cuenca           4.916860e+07
Ambato           4.030440e+07
Santo Domingo    3.583432e+07
Name: sales, dtype: float64

Top 10 product lines by total sales:
family
GROCERY I        3.434627e+08
BEVERAGES        2.169545e+08
PRODUCE          1.227047e+08
CLEANING         9.752129e+07
DAIRY            6.448771e+07
BREAD/BAKERY     4.213395e+07
POULTRY          3.187600e+07
MEATS            3.108647e+07
PERSONAL CARE    2.459205e+07
DELI             2.411032e+07
Name: sales, dtype: float64


In [4]:
# Convert the date column to datetime format
train_df['date'] = pd.to_datetime(train_df['date'])

# Extract the year and month from the date column
train_df['year_month'] = train_df['date'].dt.to_period('M')

# Filter the data for the first three months of 2016 and 2017
data_2016 = train_df[train_df['year_month'].between('2016-01', '2016-03')]
data_2017 = train_df[train_df['year_month'].between('2017-01', '2017-03')]

# Calculate the total sales for each store in the first three months of 2016 and 2017
sales_2016 = data_2016.groupby('store_nbr')['sales'].sum()
sales_2017 = data_2017.groupby('store_nbr')['sales'].sum()

# Calculate the growth rate for each store
growth_rate = (sales_2017 - sales_2016) / sales_2016

# Sort the stores based on growth rate in descending order
most_growth_rate = growth_rate.sort_values(ascending=False)
least_growth_rate = growth_rate.sort_values(ascending=True)

# Get the top 5 stores with the highest growth rate
top_5_stores = most_growth_rate.head(5)
bot_5_stores = least_growth_rate.head(5)

# Merge the sales data with the stores data based on store_nbr
merged_data = pd.merge(sales_2016, sales_2017, left_index=True, right_index=True)
merged_data = pd.merge(merged_data, store_df, left_index=True, right_on='store_nbr')

# Calculate the growth rate for each city
merged_data['growth_rate'] = (merged_data['sales_y'] - merged_data['sales_x']) / merged_data['sales_x']

# Group the data by city and calculate the average growth rate
city_growth_rate = merged_data.groupby('city')['growth_rate'].mean()

# Sort the cities based on growth rate in descending order
max_growth_rate = city_growth_rate.sort_values(ascending=False)
min_growth_rate = city_growth_rate.sort_values(ascending=True)

# Get the top 5 cities with the highest growth rate
top_5_cities = max_growth_rate.head(5)
bot_5_cities = min_growth_rate.head(5)

# Merge the sales data for 2016 and 2017 based on product family
merged_data_product = pd.merge(sales_2016, sales_2017, left_index=True, right_index=True)

# Calculate the growth rate for each product family
merged_data_product['growth_rate'] = (merged_data_product['sales_y'] - merged_data_product['sales_x']) / merged_data_product['sales_x']

# Sort the product families based on growth rate in descending order
max_growth_rate = merged_data_product['growth_rate'].sort_values(ascending=False)
min_growth_rate = merged_data_product['growth_rate'].sort_values(ascending=True)

# Get the top 10 product families with the highest growth rate
top_10_families = max_growth_rate.head(10)
bot_10_families = min_growth_rate.head(10)


# Display the top 5 stores with the highest growth rate
print(top_5_stores)
print(bot_5_stores)
print(top_5_cities)
print(bot_5_cities)
print(top_10_families)
print(bot_10_families)


store_nbr
53    0.509542
43    0.321299
42    0.230761
26    0.226196
41    0.225012
Name: sales, dtype: float64
store_nbr
13   -0.049202
16   -0.046178
19   -0.026257
33   -0.009664
10    0.009099
Name: sales, dtype: float64
city
Manta         0.509542
Esmeraldas    0.321299
Puyo          0.216018
Machala       0.177596
Daule         0.156919
Name: growth_rate, dtype: float64
city
Guaranda    -0.026257
Latacunga   -0.012098
Quevedo     -0.009664
Playas       0.009744
Libertad     0.022058
Name: growth_rate, dtype: float64
store_nbr
53    0.509542
43    0.321299
42    0.230761
26    0.226196
41    0.225012
22    0.216018
29    0.185927
32    0.185286
21    0.183100
17    0.170502
Name: growth_rate, dtype: float64
store_nbr
13   -0.049202
16   -0.046178
19   -0.026257
33   -0.009664
10    0.009099
35    0.009744
36    0.022058
12    0.025006
31    0.025782
25    0.026104
Name: growth_rate, dtype: float64


In [5]:
# Convert the date columns to datetime format
train_df['date'] = pd.to_datetime(train_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])

# Extract the year and month from the date columns
train_df['year_month'] = train_df['date'].dt.to_period('M')
oil_df['year_month'] = oil_df['date'].dt.to_period('M')

# Merge the oil prices with the sales data based on the year and month
merged_data = pd.merge(train_df, oil_df, on='year_month')
# Calculate the correlation between oil prices and sales
correlation = merged_data['sales'].corr(merged_data['dcoilwtico'])

# Display the correlation coefficient
print("Correlation between Oil Prices and Sales: ", correlation)



Correlation between Oil Prices and Sales:  -0.07495842888642766


In [6]:
# Calculate the correlation between oil prices and sales for each product family
family_correlations = merged_data.groupby('family')['sales', 'dcoilwtico'].corr().unstack()['sales']['dcoilwtico']

# Sort the correlations in descending order
sorted_correlations = family_correlations.sort_values(ascending=False)

# Get the product family with the highest correlation
most_affected_family = sorted_correlations.idxmax()
top = sorted_correlations.head(5)
# Display the most affected family
print("Most Affected Product Family: ", top)


/tmp/ipykernel_20/4219280582.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  family_correlations = merged_data.groupby('family')['sales', 'dcoilwtico'].corr().unstack()['sales']['dcoilwtico']


Most Affected Product Family:  family
LINGERIE           0.083576
HOME APPLIANCES    0.026211
MEATS             -0.026203
SEAFOOD           -0.043230
GROCERY II        -0.058283
Name: dcoilwtico, dtype: float64
